In [3]:
# importamos librerías
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv

# Cargamos variables de entorno
load_dotenv()

False

In [4]:
# endpoint de la API de datos de la Comunidad de Madrid
#url = os.getenv("API_URL")
url = "https://datos.comunidad.madrid/api/3/action/package_show?id=calidad_aire_datos_historico"

In [5]:
# Hacemos la petición y comprobamos el estado
response = requests.get(url)
response.status_code

200

In [6]:
# Convertimos a json
data = response.json()
data.keys()

dict_keys(['help', 'success', 'result'])

In [7]:
# Comprobamos que hemos obtenido un diccionario de diccionarios
type(data["result"])

dict

In [8]:
# seleccionamos el elemento del diccionario obtenido que contiene los resultados
resultado = data.get("result")
resultado.keys()

dict_keys(['license_title', 'maintainer', 'relationships_as_object', 'private', 'maintainer_email', 'num_tags', 'id', 'metadata_created', 'owner_org', 'metadata_modified', 'author', 'author_email', 'state', 'version', 'license_id', 'type', 'resources', 'num_resources', 'tags', 'groups', 'creator_user_id', 'relationships_as_subject', 'name', 'isopen', 'url', 'notes', 'title', 'extras', 'license_url', 'organization'])

In [9]:
# seleccionamos el elemento del diccionario obtenido que contiene la información de los recursos disponibles
periodo = resultado.get("resources")
type(periodo)

list

In [10]:
type(periodo[0])


dict

In [11]:
periodo[0].keys()

dict_keys(['cache_last_updated', 'cache_url', 'mimetype_inner', 'hash', 'description', 'metadata_modified', 'format', 'url', 'created', 'state', 'package_id', 'last_modified', 'mimetype', 'url_type', 'position', 'size', 'datastore_active', 'id', 'resource_type', 'name'])

In [31]:
# periodo es una lista de diccionarios. Tendremos que recorrerla para obtener los enlaces a los ficheros
nombre = ["2024","Descripción datos de contaminantes"]
for n in nombre:
    for elemento in periodo:
        if elemento["name"] == n:
            datos = elemento.get("url")
            formato = elemento.get("format")
            response = requests.get(datos)
            # Verificar si la descarga fue exitosa
            if response.status_code == 200:
             # Guardar el archivo en tu computadora
                with open(f'../datos/madrid_{n}.{formato}', 'wb') as file:
                    file.write(response.content)
                print("Archivo guardado exitosamente.")
            else:
                print(f"No se pudo descargar el archivo. Código de estado: {response.status_code}")

Archivo guardado exitosamente.
Archivo guardado exitosamente.


In [37]:
import camelot

# Ruta del archivo PDF
archivo_pdf = "../datos/madrid_Descripción datos de contaminantes.PDF"

# Extraer tablas
tablas = camelot.read_pdf(archivo_pdf, pages="3")

# Exportar a CSV o mostrar resultados
for i, tabla in enumerate(tablas):
    tabla.to_csv(f"tabla_{i + 1}.csv") 
    df_tabla = pd.read_csv(f"tabla_{i + 1}.csv") # Guardar como CSV
df_tabla # Mostrar la tabla como DataFrame


CropBox missing from /Page, defaulting to MediaBox


,CÓDIGO \nMAGNITUD,DESCRIPCIÓN MAGNITUD,CÓDIGO \nTÉCNICA \nDE MEDIDA,DESCRIPCIÓN TÉCNICA \nDE MEDIDA,UNIDAD,DESCRIPCIÓN UNIDAD
0,1,Dióxido de azufre,38,Fluorescencia ultravioleta,µg/m³,microgramos por metro cúbico
1,6,Monóxido de carbono,48,Espectrometría infrarroja no \ndispersiva,mg/m³,miligramos por metro cúbico
2,7,Monóxido de nitrógeno,8,Quimioluminiscencia,µg/m³,microgramos por metro cúbico
3,8,Dióxido de nitrógeno,8,Quimioluminiscencia,µg/m³,microgramos por metro cúbico
4,9,"Partículas en suspensión < PM2,5",49,Absorción beta,µg/m³,microgramos por metro cubico
5,10,Partículas en suspensión < PM10,49,Absorción beta,µg/m³,microgramos por metro cubico
6,11,Partículas en suspensión < PM1,49,Absorción beta,µg/m³,microgramos por metro cubico
7,12,Óxidos de nitrógeno,8,Quimioluminiscencia,µg/m³,microgramos por metro cúbico
8,14,Ozono,6,Absorción ultravioleta,µg/m³,microgramos por metro cubico
9,14,Ozono Quimioluminiscencia,8,Quimioluminiscencia,µg/m³,microgramos por metro cubico


In [ ]:
# Obtenemos los nombres de los eventos que están en curso durante las fechas de estancia
from datetime import datetime #importamos librería para transformar fechas
eventos_fechas = {"nombre_evento":[], 
                  "url_evento": [], 
                  "codigo_postal": [], 
                  "direccion": [], 
                  "horario": [],
                  "fecha_inicio": [],
                  "fecha_fin":[], 
                  "organizacion":[]
                  } #diccionario para almacenar los datos *faltan fechas inicio y fin*
#transformamos las fechas de reservas para tener solo la fecha
fecha_entrada = datetime.strptime("2025-03-01", "%Y-%m-%d").date()
fecha_salida = datetime.strptime("2025-03-02", "%Y-%m-%d").date()
# iteramos por cada elemento del diccionario
for evento in eventos:
    # transformamos la fecha para manterner el mismo formato
    fecha_inicio = datetime.strptime(evento["dtstart"], "%Y-%m-%d %H:%M:%S.%f").date()
    fecha_fin = datetime.strptime(evento["dtend"], "%Y-%m-%d %H:%M:%S.%f").date()
    if (fecha_inicio <= fecha_entrada and fecha_fin >= fecha_salida): #si el evento empieza antes o en la fecha de entrada y termina en la fecha de salida o después
        nombre = evento.get("title")
        eventos_fechas["nombre_evento"].append(nombre)
        link = evento.get("link")
        eventos_fechas["url_evento"].append(link)
        direccion_comp = evento.get("address")
        if direccion_comp !=None:
            area = evento.get("address").get("area")
            if area != None:
                codigo_postal=evento.get("address").get("area").get("postal-code")
                direccion = evento.get("address").get("area").get("street-address")
            else:
                codigo_postal= "Dirección no disponible"
                direccion = "Dirección no disponible"
        else: 
            codigo_postal= "Dirección no disponible"
            direccion = "Dirección no disponible"
        eventos_fechas["codigo_postal"].append(codigo_postal)
        eventos_fechas["direccion"].append(direccion)
        hora = evento.get("time")
        eventos_fechas["horario"].append(hora)
        eventos_fechas["fecha_inicio"].append(fecha_inicio)
        eventos_fechas["fecha_fin"].append(fecha_fin)
        organiza = evento.get("organization")
        if organiza != None:
            organizacion = evento.get("organization").get("organization-name")
        else:
            organizacion = "Organización no disponible"
        eventos_fechas["organizacion"].append(organizacion)
#Generamos un dataframe con los eventos
df_eventos = pd.DataFrame(eventos_fechas)
#Guardamos el dataframe en un archivo pickle
df_eventos.to_pickle("../data/data_clean/eventos.pkl")